In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn #remove unnecessary warnings

In [2]:
data = pd.read_csv('biddings.csv')
dfo = pd.DataFrame(data)


In [36]:
dfo = dfo.sort_values(by=['convert'])
dfo = dfo.reset_index(drop=True)
df_or = dfo.drop(dfo.index[0:992000]) #export to csv
df_or.to_csv('biddings1.csv')
# print(df_or.convert.value_counts())

# df1 = dfo.drop(dfo.index[0:996200])


In [32]:
df = df1.reset_index(drop=True) #reset index

#Convert column = shown ad was clicked: 1 = clicked ad
print(df.convert.value_counts())

df.tail()

1    1908
0    1892
Name: convert, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,convert
3795,0.04,-5.10,-2.14,0.04,1.37,-4.07,0.76,5.10,-4.58,-1.81,...,1.06,-0.51,2.46,1.03,0.43,0.59,-0.06,1.31,-0.36,1
3796,-0.00,-2.05,-1.77,0.04,1.51,-4.25,0.39,4.84,-4.50,-2.68,...,1.63,-0.40,-0.09,0.16,1.66,0.16,-0.08,-0.00,0.05,1
3797,-0.01,0.71,2.30,-0.23,-0.67,-0.05,0.24,0.11,0.43,0.55,...,-0.19,0.39,1.06,0.05,-0.87,-0.01,1.79,-0.76,0.22,1
3798,-0.02,2.15,-0.83,0.11,1.56,-4.30,0.34,4.17,-4.27,-3.92,...,1.63,0.43,0.11,-0.07,0.03,-0.00,-0.05,-0.12,0.09,1
3799,-0.01,-0.76,-4.40,-0.19,-0.76,2.45,0.92,-0.50,-0.18,0.01,...,0.01,-1.22,0.24,0.22,0.31,0.10,0.13,-0.17,0.08,1


In [33]:
from sklearn.model_selection import train_test_split

x = df.drop(['convert'],axis=1) #training features
y = df.convert #target

#Set our train/test values
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                  test_size = .10)
#Test sample
y_test.value_counts()


0    201
1    179
Name: convert, dtype: int64

In [34]:
y_train.value_counts()


1    1729
0    1691
Name: convert, dtype: int64

In [35]:
#First model using our undersampled data
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

clf = LinearSVC(random_state=42)
clf.fit(x_train, y_train) 

print('Training accuracy LinearSVC is: %0.2f' % (clf.score(x_train, y_train)))
print('Test accuracy LinearSVC is: %0.2f' % (clf.score(x_test, y_test)))

scores = cross_val_score(clf, x_train, y_train, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

print('\nThis LinearSVC runs at a speed of:')
%timeit clf.predict(df.iloc[5:6, :88])

Training accuracy LinearSVC is: 0.66
Test accuracy LinearSVC is: 0.63
Cross Val LinearSVC Accuracy: 0.64 (+/- 0.04)

              precision    recall  f1-score   support

           0       0.66      0.64      0.65       201
           1       0.61      0.63      0.62       179

   micro avg       0.63      0.63      0.63       380
   macro avg       0.63      0.63      0.63       380
weighted avg       0.64      0.63      0.63       380


This LinearSVC runs at a speed of:
1.28 ms ± 95.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
from sklearn.metrics import f1_score

#Show f1 score 
print("LinearSVC F1 Score: %0.2f \n" % f1_score(y_test, y_pred))


LinearSVC F1 Score: 0.75 



### Create new csv with new df as the dataset